In [8]:
import sys
from pathlib import Path

# Ubicación del notebook
NOTEBOOK_DIR = Path.cwd()

# Raíz del proyecto = subir un nivel desde Notebooks/
PROJECT_ROOT = NOTEBOOK_DIR.parent

# Añadir raíz del proyecto al sys.path
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print("Proyecto raíz detectado:", PROJECT_ROOT)


Proyecto raíz detectado: c:\Proyectos\EduFinance\EduFinance_Simulator


In [10]:
"""Importamos las librerias a usar"""

import pandas as pd
import numpy as np
import yfinance as yf
from functools import lru_cache
import yaml
import os
from utils.paths import BASE_DIR, DATA_DIR, FIG_DIR
from utils.loader import save_csv, save_yaml


print(f"Carpeta '{DATA_DIR}' lista.")
print(f"Carpeta '{FIG_DIR}' lista.")

Carpeta 'C:\Proyectos\EduFinance\EduFinance_Simulator\data' lista.
Carpeta 'C:\Proyectos\EduFinance\EduFinance_Simulator\figures' lista.


In [11]:
"""Definición de tickers y fechas """

TICKERS_YF = [
    "VOO",      # Vanguard S&P 500 ETF (EE. UU., 500 grandes empresas)
    "QQQ",      # Invesco Nasdaq-100 ETF (tecnología/crecimiento)
    "EUNL.DE",  # iShares MSCI World (Xetra, EUR; exposición global desarrollados)
    "XAR",      # SPDR Aerospace & Defense ETF (sector defensa)
    "TSLA",     # Tesla Inc. (vehículos eléctricos / energía)
    "V",        # Visa Inc. (pagos electrónicos)
    "BTC-USD",  # Bitcoin en USD (cripto 24/7)
    "XRP-USD"   # XRP en USD (cripto 24/7)
]

START_DATE = "2015-01-02"  # fecha de inicio fija acordada
END_DATE   = "2025-09-05"  # fecha de corte fija acordada

print("\nConstantes definidas.")
print("Tickers:", TICKERS_YF)
print("Rango:", START_DATE, "→", END_DATE)



Constantes definidas.
Tickers: ['VOO', 'QQQ', 'EUNL.DE', 'XAR', 'TSLA', 'V', 'BTC-USD', 'XRP-USD']
Rango: 2015-01-02 → 2025-09-05


In [12]:
@lru_cache(maxsize=None)
def _get_meta_from_yf(tk: str) -> dict:
    """
    Función: _get_meta_from_yf
    -------------------------------------
    Obtiene metadatos básicos de un ticker financiero desde Yahoo Finance.

    Propósito
    --------
    - Determinar de manera automática:
        • 'asset_class' → Clase de activo (ej.: Stock, ETF, Crypto, etc.)
        • 'currency'    → Moneda principal de cotización (ej.: USD, EUR).
    - Utilizar primero métodos rápidos (fast_info) y, en caso de no encontrar
      la información, usar métodos más completos (info).
    - Proporcionar **reglas de respaldo (heurísticas)** para casos en los que
      Yahoo Finance no devuelve la información esperada.

    Parámetros
    ----------
    tk : str
        Símbolo del activo en Yahoo Finance.
        Ejemplos:
        - "VOO"       → Vanguard S&P 500 ETF
        - "TSLA"      → Tesla Inc.
        - "BTC-USD"   → Bitcoin en USD
        - "EUNL.DE"   → iShares MSCI World en Xetra (EUR)

    Retorna
    -------
    dict
        Diccionario con dos claves:
        {
            "asset_class": str, #clase del activo (Stock, ETF, Crypto, Unknown...)
            "currency": str     #moneda principal de negociación (USD, EUR, ...)
        }

    Detalles de implementación
    --------------------------
    1. Intento inicial con `fast_info`
       - Es un acceso rápido proporcionado por `yfinance`.
       - Extraemos `currency` y `quoteType` si están disponibles.
       - Mapeamos `quoteType` a una clase de activo conocida.

    2. Intento secundario con `info`
       - Más costoso en tiempo y puede fallar.
       - Usado solo si no se obtuvieron valores de `fast_info`.

    3. Reglas de respaldo (heurísticas)
       - Si el ticker tiene formato `XXX-USD`, `XXX-EUR`, etc. → se asume "Crypto".
       - Si termina en `.DE` (mercado Xetra, Alemania) → se asigna "EUR" como moneda.
       - Otros patrones podrían añadirse según se amplíe el proyecto.

    4. Valores por defecto
       - Si no se logra inferir el tipo de activo → "Unknown".
       - Si no se logra inferir la moneda → "USD".

    Decorador
    ---------
    @lru_cache(maxsize=None)
    - Memoriza resultados en caché.
    - Evita consultas repetidas a Yahoo Finance para el mismo ticker,
      mejorando la eficiencia.

    """
    #Se instancia el objeto tickerde yfinance para el simbolo solicitado
    t = yf.Ticker(tk)

    #Inicializamos las variables
    asset_class = None
    currency    = None

    # 1) Hacemos un intento rapido cons fast_info
    try:
        fi = getattr(t, "fast_info", None) or {}
        if hasattr(fi, "__dict__"):
            fi = fi.__dict__
        #Moneda de cotizacion
        currency    = fi.get("currency", None)
        #Tipo de instrumento (si esta disponible): mapeamos la clase
        quote_type  = fi.get("quoteType", None)
        if quote_type:
          asset_class = {
              "EQUITY": "Stock",
              "ETF": "ETF",
              "MUTUAL_FUND": "Mutual Fund",
              "CRYPTOCURRENCY": "Crypto",
              "INDEX": "Index",
          }.get(str(quote_type).upper(), None)
    except Exception: pass

    # 2) Intento completo con info (más costoso y propenso a fallos)
    #  - Sólo se consulta si aún faltan datos (asset_class o currency).
    if asset_class is None or currency is None:
        try:
            info = t.info  # Puede tardar / lanzar excepciones según el ticker
            if currency is None:
                currency = info.get("currency", None)
            if asset_class is None:
                qt = info.get("quoteType", None)
                if qt:
                    asset_class = {
                        "EQUITY": "Stock",
                        "ETF": "ETF",
                        "CRYPTOCURRENCY": "Crypto",
                        "MUTUALFUND": "Fund",
                        "INDEX": "Index",
                    }.get(str(qt).upper(), None)
        except Exception: pass


    # 3) Heurísticas de respaldo
    #  - Si no pudimos inferir la clase/moneda por API, deducimos por patrón.

    if asset_class is None:
        # Patrones típicos de tickers cripto: 'XXX-USD', 'XXX-EUR', 'XXX-USDT', etc.
        if "-" in tk and tk.split("-")[-1] in {"USD", "EUR", "USDT", "BTC", "ETH"}:
            asset_class = "Crypto"

    if currency is None:
        # Si el ticker contiene '-', el segmento final suele ser la divisa
        # (p. ej., 'BTC-USD' → 'USD').
        if "-" in tk:
            maybe_ccy = tk.split("-")[-1]
            # Códigos ISO típicos de 3 letras (o 4 en el caso de USDT)
            if len(maybe_ccy) in (3, 4):
                currency = maybe_ccy

        # Sufijo de bolsa alemana (.DE) → eurístico a EUR (Xetra)
        if currency is None and tk.endswith(".DE"):
            currency = "EUR"

    # 4) Valores por defecto si no se pudo inferir
    asset_class = asset_class or "Unknown"
    currency = currency or "USD"

    # Retornamos los metadatos consolidados
    return {"asset_class": asset_class, "currency": currency}

In [13]:
def download_yf(tickers, start, end) -> pd.DataFrame:
    """
    Descarga diaria (precios ajustados) desde Yahoo Finance para múltiples tickers,
    devolviendo un DataFrame en formato 'tidy'.
    """
    import yfinance as yf
    import pandas as pd

    raw = yf.download(
        tickers=tickers,
        start=start,
        end=end,
        interval="1d",
        auto_adjust=True,
        group_by="ticker",
        threads=True
    )

    frames = []

    for tk in tickers:
        try:
            df_tk = raw[tk].copy()
        except Exception:
            df_tk = raw.copy()

        # Verificamos columnas válidas
        cols = [c for c in df_tk.columns if c.lower() in ("close", "adj close")]
        if not cols:
            print(f"⚠️ No se encontró columna de cierre para {tk}. Ticker omitido.")
            continue

        # Renombramos a 'close'
        df_tk = df_tk[cols].rename(columns={"Adj Close": "close", "Close": "close"})

        # Aseguramos la columna de fecha
        df_tk = df_tk.copy()
        df_tk = df_tk.reset_index(names="date") if df_tk.index.name else df_tk.reset_index()
        if "Date" in df_tk.columns:
            df_tk = df_tk.rename(columns={"Date": "date"})
        if "index" in df_tk.columns:
            df_tk = df_tk.rename(columns={"index": "date"})

        # Llamamos a la función auxiliar de metadatos
        meta = _get_meta_from_yf(tk)

        # Armamos el DataFrame final por ticker
        df_tk["ticker"] = tk
        df_tk["asset_class"] = meta["asset_class"]
        df_tk["currency"] = meta["currency"]
        df_tk = df_tk[["date", "ticker", "asset_class", "close", "currency"]]

        frames.append(df_tk)

    # Eliminamos posibles DataFrames vacíos
    frames = [f for f in frames if not f.empty]
    if not frames:
        raise RuntimeError("\nNingún ticker produjo datos válidos. Revisa los símbolos o las fechas.")

    # Concatenamos todo con seguridad
    out = pd.concat(frames, ignore_index=True, axis=0)
    out["date"] = pd.to_datetime(out["date"], errors="coerce").dt.date
    out = (
        out.dropna(subset=["date"])
           .sort_values(["ticker", "date"])
           .drop_duplicates()
           .reset_index(drop=True)
    )

    # Sanity checks
    if out.empty:
        raise RuntimeError("\nLa descarga produjo un DataFrame vacío. Verifica tickers o conexión.")
    if out["close"].isna().mean() > 0.5:
        print("\nMás del 50% de 'close' es NaN; revisa conexión o símbolos.")

    return out


# Ejemplo de ejecución
market_df = download_yf(TICKERS_YF, START_DATE, END_DATE)

[*********************100%***********************]  8 of 8 completed


In [15]:
"""
Guardamos los resultados 
"""

save_csv(market_df, "raw_market_data.csv")

save_yaml(
    {
        "tickers": TICKERS_YF,
        "start_date": START_DATE,
        "end_date": END_DATE,
        "n_rows": len(market_df),
        "n_tickers": len(market_df["ticker"].unique())
    },
    "metadata.yml"
)

print("Datos y metadata guardados en /data")

Archivo guardado en: C:\Proyectos\EduFinance\EduFinance_Simulator\data\raw_market_data.csv
YAML guardado en: C:\Proyectos\EduFinance\EduFinance_Simulator\metadata.yml
Datos y metadata guardados en /data


In [16]:
"""
Sanity check: Cobertura temporal por ticker

El objetivo de este bloque es verificar que la descarga de datos
desde Yahoo Finance fue correcta para cada activo.

1. Agrupamos por 'ticker'.
2. Calculamos:
  - first_date: primera fecha disponible en la serie.
  - last_date : última fecha disponible.
  - n_rows    : cantidad total de observaciones (filas).
  - n_missing : cantidad de valores nulos en 'close'.
3. Ordenamos los resultados por ticker para facilitar la inspección.

Este chequeo es importante porque:
  - Nos ayuda a identificar si algún activo tiene un rango temporal
    mucho más corto que el resto (ej. XRP con datos recientes).
  - Detecta gaps grandes o descargas vacías.
  - Nos da un panorama rápido de la consistencia antes de pasar
  a la normalización de fechas.

"""

checks = (
    market_df
    .groupby("ticker")
    .agg(
        first_date=("date", "min"),   # primera fecha disponible
        last_date=("date", "max"),    # última fecha disponible
        n_rows=("date", "count"), # cantidad de observaciones
        n_missing = ("close", lambda s: s.isna().sum()), # nulos en 'close'
        asset_class = ("asset_class", "first"), # clase del activo
        currency = ("currency", "first")
    )
    .sort_index()
)

checks

,first_date,last_date,n_rows,n_missing,asset_class,currency
ticker,,,,,,
BTC-USD,2015-01-02,2025-09-04,3899,0,Crypto,USD
EUNL.DE,2015-01-02,2025-09-04,3899,1184,ETF,EUR
QQQ,2015-01-02,2025-09-04,3899,1215,ETF,USD
TSLA,2015-01-02,2025-09-04,3899,1215,Stock,USD
V,2015-01-02,2025-09-04,3899,1215,Stock,USD
VOO,2015-01-02,2025-09-04,3899,1215,ETF,USD
XAR,2015-01-02,2025-09-04,3899,1215,ETF,USD
XRP-USD,2015-01-02,2025-09-04,3899,1042,Crypto,USD
